In [ ]:
"""
Created on Tue Jul 6 11:08 2021

Prepare the results for spatial plots

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import assess_param_funcs.useful_functions as uf
from assess_param_funcs.constants import *

import distributed

import matplotlib as mpl
import glob

In [ ]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ IN DATA

In [ ]:
nemo_run = 'OPM021'

if nemo_run == 'OPM006':
    yy_start = 1989
    yy_end = 2018
elif nemo_run == 'OPM021':
    yy_start = 1989
    yy_end = 2018
elif nemo_run == 'OPM016' or nemo_run == 'OPM018':
    yy_start = 1980
    yy_end = 2008

In [ ]:
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
path_simple = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
path_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
path_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/'
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'

In [ ]:
file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

file_conc = xr.open_mfdataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)#.chunk(chunks={'x': chunk_size, 'y': chunk_size})

In [ ]:
isf_mask = (file_isf['ISF_mask'] == file_isf.Nisf)

In [ ]:
xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)

Time mean spatial

In [ ]:
NEMO_melt_rates_2D = xr.open_mfdataset(outputpath_melt+'melt_rates_2D_NEMO.nc')

In [ ]:
NEMO_melt_rates_2D_timmean = NEMO_melt_rates_2D.mean('time')
NEMO_melt_rates_2D_timmean.to_netcdf(outputpath_melt+'melt_rates_2D_NEMO_timmean.nc')

PREPARE PARAM FILES

In [ ]:
melt_param_files = list(sorted(glob.glob(outputpath_melt+'melt_rates_2D_*_correctedTS_oneFRIS.nc')))

param_list = []
file_list = []

for mfilename in melt_param_files:
    if ('favier' not in mfilename) and ('lipscomb' not in mfilename):
        paramname = mfilename[97:157].split('.')[0].split('_')[:-2]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
        file_list.append(mfilename)

In [ ]:
param_list

In [ ]:
ds_melt_param  = xr.open_mfdataset(file_list, concat_dim='new_param', combine='nested', coords='minimal',compat='override', chunks={'x': 200, 'y': 200})
ds_melt_param = ds_melt_param.drop('param')#.drop('calib')#.drop('option')
ds_melt_param = ds_melt_param.rename({'new_param': 'param'})
ds_melt_param = ds_melt_param.assign_coords(param=param_list)

In [ ]:
#aram_simple_list = ['linear_local_favier', 'linear_local_tuned', 'quadratic_local_favier', 'quadratic_local_lipscomb', 'quadratic_local_tuned', 
#                     'quadratic_mixed_cavslope_tuned','quadratic_mixed_locslope_lipscomb', 'quadratic_mixed_locslope_tuned', 'quadratic_mixed_mean_favier', 
#                     'quadratic_mixed_mean_lipscomb', 'quadratic_mixed_mean_tuned']
#param_simple_list = ['linear_local_tuned', 'quadratic_local_tuned', 'quadratic_local_locslope_tuned', 'quadratic_local_cavslope_tuned',
#                     'quadratic_mixed_cavslope_tuned','quadratic_mixed_locslope_tuned', 'quadratic_mixed_mean_tuned']
param_simple_list = ['quadratic_local_tuned','quadratic_mixed_mean_tuned','quadratic_mixed_locslope_tuned']

In [ ]:
param_plume_list = []
for opt_par in ['lazero19_2','lazero19_modif2']:
    for tune_mode in ['tuned']: #'orig'
        param_plume_list.append(str(opt_par)+'_'+tune_mode)

In [ ]:
opt_boxes = ['pismno_picopno','pismyes_picopno','pismno_picopyes','pismyes_picopyes']
param_box_list = [ ]
for opt in opt_boxes:
    for tune_mode in ['tuned_clara']:
        for bb_nb in range(1,4):
            param_box_list.append('boxes_'+str(bb_nb)+'_'+opt+'_'+tune_mode)

In [ ]:
ds_melt_param['melt_m_ice_per_y'].param

MAKE TIME MEANS FOR EACH PARAM FAMILY

In [ ]:
ds_melt_m_ice_per_y_simple_timmean = ds_melt_param['melt_m_ice_per_y'].sel(param=param_simple_list).mean('time')
ds_melt_m_ice_per_y_simple_timmean.to_netcdf(outputpath_melt+'melt_rates_2D_simple_timmean_oneFRIS.nc')


In [ ]:
ds_melt_m_ice_per_y_plumes_timmean = ds_melt_param['melt_m_ice_per_y'].sel(param=param_plume_list).mean('time')
ds_melt_m_ice_per_y_plumes_timmean.to_netcdf(outputpath_melt+'melt_rates_2D_plumes_timmean_oneFRIS.nc')

In [ ]:
param_box_list = ['boxes_1_pismyes_picopno_tuned_clara',
       'boxes_2_pismyes_picopno_tuned_clara',
       'boxes_3_pismyes_picopno_tuned_clara', 
       'boxes_3_pismyes_picopyes_tuned_clara',
       'boxes_4_pismno_picopyes_tuned_clara',
       'boxes_4_pismyes_picopno_tuned_clara']

In [ ]:
ds_melt_m_ice_per_y_plumes_timmean = ds_melt_param['melt_m_ice_per_y'].sel(param=param_box_list).mean('time')
ds_melt_m_ice_per_y_plumes_timmean.to_netcdf(outputpath_melt+'melt_rates_2D_boxes_timmean_oneFRIS.nc')